## Task:
1. Try to quantize the full model, run on data, check the difference in the output wrt. the fp32 model.
2. Try to quantize the full model, run on data, check the difference in the output wrt. the fp16 model.
3. Try to quantize the full model, run on data, check the difference in the output wrt. the int8 model.
4. what matters is if you quantize a layer or the whole network, how does the inference runtime of the network change?
5. measuring the inference throughput (time per batch)? that would be the relevant metric, along with the error in the output compared to the fp32 version
6. run the inference on a bunch of real events from the dataset, and check the error introduced by quantization in the inference compared to fp32. and also measure the inference time
8. To check the error in the ouptu, do we need to use all of the quantized weights or a few of them?
9. What is the role of the .pkl file?


In [9]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import h5py

import tensorflow as tf
import tensorflow_datasets as tfds


2023-10-23 17:05:57.254338: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-23 17:05:57.449526: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-23 17:05:57.449564: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-23 17:05:57.451539: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-23 17:05:57.546974: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-23 17:05:57.548277: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

The given datasets are tensorflow datasets, which can be loaded as 
```python
import tensorflow as tf
import tensorflow_datasets as tfds

ds_builder = tfds.builder("clic_edm_qq_pf")
dss = ds_builder.as_data_source("test")

for elem in dss:
    print({"X": elem["X"], "ygen": elem["ygen"], "ycand": elem["ycand"]})
```
X is each event that you can feed to the model



In [2]:
 '../../tensorflow_datasets/clic_edm_qq_pf/1.5.0/clic_edm_qq_pf-test.array_record-00000-of-00032'

In [31]:
import tensorflow as tf
import tensorflow_datasets as tfds

# Define the feature description for parsing
feature_description = {
    'X': tf.io.FixedLenFeature([], tf.string),
    'ygen': tf.io.FixedLenFeature([], tf.string),
    'ycand': tf.io.FixedLenFeature([], tf.string),
}

# Define a function to parse the TFRecord
def parse_tfrecord_fn(example):
    example = tf.io.parse_single_example(example, feature_description)
    return example

# Load the dataset
data_dir = '../../tensorflow_datasets/clic_edm_qq_pf-test.array_record-'
files = [f'{data_dir}{i:05d}-of-00032' for i in range(32)]
raw_dataset = tf.data.TFRecordDataset(files)
parsed_dataset = raw_dataset.map(parse_tfrecord_fn)

# Iterate through the dataset
for elem in parsed_dataset:
    X = tf.io.decode_raw(elem['X'], tf.float32)
    ygen = tf.io.decode_raw(elem['ygen'], tf.float32)
    ycand = tf.io.decode_raw(elem['ycand'], tf.float32)

NotFoundError: {{function_node __wrapped__IteratorGetNext_output_types_3_device_/job:localhost/replica:0/task:0/device:CPU:0}} ../../tensorflow_datasets/clic_edm_qq_pf-test.array_record-00000-of-00032; No such file or directory [Op:IteratorGetNext] name: 

In [30]:
raw_dataset

<TFRecordDatasetV2 element_spec=TensorSpec(shape=(), dtype=tf.string, name=None)>